## Import libraries

In [1]:
from os import link
import os
from unicodedata import name
import logging
import json
import scrapy
from scrapy.crawler import CrawlerProcess
import json
import pandas as pd

# import files csv

In [6]:
df_info = pd.read_csv("../04_datasets/games_info_features.csv")
df_scores = pd.read_csv("../04_datasets/scores_prep.csv", header= 1)

In [7]:
display(df_info.columns)
df_scores.columns

Index(['Unnamed: 0', 'idgame', 'date', 'stade', 'location', 'attendance',
       'capacity', 'people', 'line', 'over_under', 'attendance_info', 'month',
       'dayofmonth', 'dayofweek', 'hour', 'minute'],
      dtype='object')

Index(['Unnamed: 0', 'season', 'week', 'awayteam', 'hometeam', 'awayscore',
       'homescore', 'idgame', 'score_abs', 'winner_home', 'winner_away'],
      dtype='object')

In [8]:
df_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1294 entries, 0 to 1293
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Unnamed: 0       1294 non-null   int64  
 1   idgame           1294 non-null   int64  
 2   date             1294 non-null   object 
 3   stade            1294 non-null   object 
 4   location         1294 non-null   object 
 5   attendance       1144 non-null   float64
 6   capacity         1294 non-null   int64  
 7   people           1144 non-null   float64
 8   line             528 non-null    object 
 9   over_under       474 non-null    object 
 10  attendance_info  1294 non-null   int64  
 11  month            1294 non-null   int64  
 12  dayofmonth       1294 non-null   int64  
 13  dayofweek        1294 non-null   int64  
 14  hour             1294 non-null   int64  
 15  minute           1294 non-null   int64  
dtypes: float64(2), int64(9), object(5)
memory usage: 161.9+ KB


In [12]:
df_info['idgame'] = df_info['idgame'].apply(lambda x : str(x))

In [13]:
# merge files
df_scores_info = df_scores.merge(df_info, on = 'idgame')
display(df_scores_info)

,Unnamed: 0_x,season,week,awayteam,hometeam,awayscore,homescore,idgame,score_abs,winner_home,...,capacity,people,line,over_under,attendance_info,month,dayofmonth,dayofweek,hour,minute
0,0,2018,2,Ravens,Bengals,23,34,401030690,11,1,...,65515,76.0,NaN,NaN,1,9,14,4,0,20
1,1,2018,2,Panthers,Falcons,24,31,401030691,7,1,...,75000,97.0,NaN,NaN,1,9,16,6,17,0
2,2,2018,2,Chargers,Bills,31,20,401030704,-11,0,...,71621,97.0,NaN,NaN,1,9,16,6,17,0
3,3,2018,2,Texans,Titans,17,20,401030694,3,1,...,69143,90.0,NaN,NaN,1,9,16,6,17,0
4,4,2017,2,Texans,Bengals,13,9,400951620,-4,0,...,65515,81.0,NaN,NaN,1,9,15,4,0,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1284,1284,2021,18,49ers,Rams,27,24,401326599,-3,0,...,71500,100.0,Line: LAR -3.0,Over/Under: 46.5,1,1,9,6,21,25
1285,1285,2021,18,Patriots,Dolphins,24,33,401326592,9,1,...,64767,100.0,Line: NE -6.0,Over/Under: 41.0,1,1,9,6,21,25
1286,1286,2021,18,Seahawks,Cardinals,38,30,401326597,-8,0,...,65000,98.0,Line: ARI -5.5,Over/Under: 48.5,1,1,9,6,21,25
1287,1287,2021,18,Panthers,Buccaneers,17,41,401326596,24,1,...,65828,99.0,Line: TB -11.0,Over/Under: 43.0,1,1,9,6,21,25


In [14]:
import datetime

In [15]:
df = df_scores_info.copy()

In [16]:
df = df_scores_info.copy()
# transform object in datetime
df['date'] = df['date'].apply(lambda x : datetime.datetime.fromisoformat(x))
# create a dataframe empty
df_time_teams = pd.DataFrame(columns=(['team','idgame','date',"delta_match"]))

teams = df["hometeam"].unique()

for team in teams:

    # work by home/away
    mask_home = df["hometeam"]==team
    mask_away = df["awayteam"]==team
    
    df_home = df.loc[mask_home,['hometeam','idgame','date']]
    df_away = df.loc[mask_away,['awayteam','idgame','date']]
    
    # rename column
    df_home.rename(columns={'hometeam':'team'}, inplace = True)
    df_away.rename(columns={'awayteam':'team'}, inplace = True)

    # concat away&home in one column team
    df_time = pd.concat([df_home,df_away])
    df_time.sort_values(by=['date'], ascending = True, inplace = True)

    # creat column
    df_time["delta_match"] =pd.Series()

    # calculate the time between two matches
    for index in range(len(df_time)):
        if index == 0:
            df_time["delta_match"].iloc[index] = None
        else:
            df_time["delta_match"].iloc[index] = df_time["date"].iloc[index] - df_time["date"].iloc[index-1]

    # concat result
    df_time_teams = pd.concat([df_time_teams, df_time],axis=0, ignore_index= True)
    
     

C:\Users\sylva\AppData\Local\Temp\ipykernel_14252\672681597.py:27: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df_time["delta_match"] =pd.Series()
C:\Users\sylva\AppData\Local\Temp\ipykernel_14252\672681597.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_time["delta_match"].iloc[index] = None
C:\Users\sylva\AppData\Local\Temp\ipykernel_14252\672681597.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_time["delta_match"].iloc[index] = df_time["date"].iloc[index] - df_time["date"].iloc[inde

In [17]:
# Save file

df_time_teams.to_csv('../04_datasets/games_timedelta.csv') 

In [18]:
df_time_teams.describe()

C:\Users\sylva\AppData\Local\Temp\ipykernel_14252\930267902.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  df_time_teams.describe()


,team,idgame,date,delta_match
count,2578,2578,2578,2546
unique,32,1289,542,232
top,Buccaneers,400951572,2020-09-20 17:00:00+00:00,7 days 00:00:00
freq,81,2,20,712
first,NaN,NaN,2017-09-08 00:30:00+00:00,NaN
last,NaN,NaN,2022-01-10 01:20:00+00:00,NaN


In [19]:
len(df_time_teams)

2578